# Module 03: Percentages, Ratios, and Changes

**Difficulty**: ⭐ (Beginner)

**Estimated Time**: 45-50 minutes

**Prerequisites**: 
- Module 00: Introduction and Stock Returns
- Module 01: Averages and Central Tendency
- Module 02: Spread and Variation
- Basic understanding of percentages

## Learning Objectives

By the end of this notebook, you will be able to:
1. Calculate **percentage changes** and understand their properties
2. Compute **Rate of Change (ROC)** - a key momentum indicator
3. Calculate and interpret **ratios** in trading context
4. **Normalize values** to a 0-100 scale (foundation for RSI, Stochastic)
5. Understand the **mathematical foundation of RSI** indicator

## Why This Matters

**Percentages and normalization are EVERYWHERE in technical analysis.**

Almost every oscillator indicator uses these concepts:
- **RSI (Relative Strength Index)** → Normalizes gain/loss ratio to 0-100
- **Stochastic Oscillator** → Normalizes price position to 0-100
- **Williams %R** → Normalizes price position to -100 to 0
- **ROC (Rate of Change)** → Percentage change over time
- **Money Flow Index** → Volume-weighted RSI

After this module, you'll understand the math that powers these indicators!

---

## Setup

Let's import libraries and download Malaysian stock data.

In [ ]:
# Data manipulation and numerical operations
import pandas as pd
import numpy as np

# Data acquisition
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.precision', 4)

# Random seed for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully!")

In [ ]:
# Download Malaysian stock data
print("Downloading Malaysian stock data...\n")

# Maybank - stable banking stock
maybank = yf.download('1155.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Top Glove - volatile healthcare stock
topglove = yf.download('5225.KL', start='2023-01-01', end='2024-01-01', progress=False)

# CIMB - another banking stock for comparison
cimb = yf.download('1023.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Validate data
assert len(maybank) > 0, "Failed to download Maybank data"
assert len(topglove) > 0, "Failed to download Top Glove data"
assert len(cimb) > 0, "Failed to download CIMB data"

print(f"✓ Maybank: {len(maybank)} days")
print(f"✓ Top Glove: {len(topglove)} days")
print(f"✓ CIMB: {len(cimb)} days")
print("\nData ready for analysis!")

---

## Part 1: Percentage Changes - The Foundation

### What is a Percentage Change?

**Percentage change** measures how much something changed relative to its original value.

### Formula

$$
\text{Percentage Change} = \frac{\text{New Value} - \text{Old Value}}{\text{Old Value}} \times 100\%
$$

### Why Percentages Are Essential

1. **Standardized comparison** - 5% gain means the same for all stocks
2. **Scale-independent** - Works for RM 1 or RM 100 stocks
3. **Additive properties** - Can be averaged and compared
4. **Universal language** - All traders understand percentages

In [ ]:
# Simple example: Stock price changes
old_price = 8.50
new_price = 8.93

# Calculate absolute change
absolute_change = new_price - old_price

# Calculate percentage change
percentage_change = ((new_price - old_price) / old_price) * 100

print("=" * 60)
print("PERCENTAGE CHANGE CALCULATION")
print("=" * 60)
print(f"\nOld price: RM {old_price:.2f}")
print(f"New price: RM {new_price:.2f}")
print(f"\nAbsolute change: RM {absolute_change:.2f}")
print(f"Percentage change: {percentage_change:.2f}%")
print(f"\nFormula: ({new_price:.2f} - {old_price:.2f}) / {old_price:.2f} × 100")
print(f"       = {absolute_change:.2f} / {old_price:.2f} × 100")
print(f"       = {percentage_change:.2f}%")

### Properties of Percentage Changes

Percentage changes have important mathematical properties you must understand.

In [ ]:
# Property 1: NOT symmetric
# Going up 10% then down 10% does NOT return to original price

starting_price = 100.00

# Up 10%
after_up = starting_price * (1 + 0.10)
print("Property 1: Percentage changes are NOT symmetric")
print(f"\nStarting price: RM {starting_price:.2f}")
print(f"After +10%:     RM {after_up:.2f}")

# Down 10%
after_down = after_up * (1 - 0.10)
print(f"After -10%:     RM {after_down:.2f}")
print(f"\nNet change: RM {after_down - starting_price:.2f} ({((after_down - starting_price) / starting_price) * 100:.1f}%)")
print("\n→ You LOST RM 1.00 even though gains/losses were equal!")
print("→ This is why recovering from losses requires LARGER percentage gains")

In [ ]:
# Property 2: Loss requires larger gain to recover
print("Property 2: Recovering from losses requires larger percentage gains\n")
print("=" * 70)

losses = [10, 20, 30, 40, 50, 60, 70, 80, 90]
results = []

for loss_pct in losses:
    # Calculate price after loss
    price_after_loss = 100 * (1 - loss_pct/100)
    
    # Calculate gain needed to recover
    gain_needed = ((100 - price_after_loss) / price_after_loss) * 100
    
    results.append((loss_pct, gain_needed))
    print(f"Loss of {loss_pct:2d}% requires gain of {gain_needed:6.1f}% to recover")

print("\n" + "=" * 70)
print("Key Insight: A 50% loss requires a 100% gain to break even!")
print("This is why risk management is crucial in trading.")

In [ ]:
# Visualize the asymmetry
plt.figure(figsize=(12, 6))

losses_pct = [r[0] for r in results]
gains_needed = [r[1] for r in results]

plt.plot(losses_pct, gains_needed, linewidth=3, marker='o', markersize=8, color='red')
plt.plot(losses_pct, losses_pct, linewidth=2, linestyle='--', color='green', 
         label='If gains = losses (hypothetical)', alpha=0.7)

plt.title('Asymmetry of Percentage Changes: Recovery Difficulty', 
          fontsize=14, fontweight='bold')
plt.xlabel('Loss Percentage (%)', fontsize=12)
plt.ylabel('Gain Needed to Recover (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper left', fontsize=10)

# Highlight the 50% loss point
plt.axvline(x=50, color='orange', linestyle=':', linewidth=2, alpha=0.5)
plt.axhline(y=100, color='orange', linestyle=':', linewidth=2, alpha=0.5)
plt.scatter([50], [100], s=200, color='orange', zorder=5, edgecolors='black', linewidths=2)
plt.text(50, 110, '50% loss → 100% gain needed', ha='center', fontsize=10, 
         fontweight='bold', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print("The gap between the lines shows the difficulty of recovery.")
print("As losses get larger, recovery becomes exponentially harder!")

---

## Part 2: Rate of Change (ROC) - Momentum Measurement

### What is ROC?

**Rate of Change (ROC)** measures the percentage change over a specific period.

### Formula

$$
ROC_n = \frac{\text{Price}_{today} - \text{Price}_{n\text{ days ago}}}{\text{Price}_{n\text{ days ago}}} \times 100\%
$$

Where $n$ is the lookback period (commonly 10, 20, or 30 days).

### How Traders Use ROC

1. **Momentum indicator** - Positive ROC = upward momentum
2. **Overbought/Oversold** - Extreme ROC values signal potential reversals
3. **Divergence** - When price and ROC move in opposite directions
4. **Zero-line crossovers** - ROC crossing zero signals trend changes

In [ ]:
# Calculate ROC for Maybank
maybank_close = maybank['Close']

# Calculate 10-day ROC manually
lookback = 10
maybank_roc_10 = ((maybank_close - maybank_close.shift(lookback)) / maybank_close.shift(lookback)) * 100

# Calculate other common ROC periods
maybank_roc_20 = ((maybank_close - maybank_close.shift(20)) / maybank_close.shift(20)) * 100
maybank_roc_30 = ((maybank_close - maybank_close.shift(30)) / maybank_close.shift(30)) * 100

print("Rate of Change (ROC) for Maybank")
print("\nLast 10 values of 10-day ROC:")
print(maybank_roc_10.tail(10))

print(f"\nCurrent ROC values:")
print(f"  10-day ROC: {maybank_roc_10.iloc[-1]:.2f}%")
print(f"  20-day ROC: {maybank_roc_20.iloc[-1]:.2f}%")
print(f"  30-day ROC: {maybank_roc_30.iloc[-1]:.2f}%")

In [ ]:
# Visualize ROC over time
plt.figure(figsize=(14, 10))

# Subplot 1: Price
plt.subplot(3, 1, 1)
plt.plot(maybank_close.index, maybank_close.values, linewidth=1.5, color='blue')
plt.title('Maybank Price (2023)', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

# Subplot 2: 10-day ROC
plt.subplot(3, 1, 2)
plt.plot(maybank_roc_10.index, maybank_roc_10.values, linewidth=1.5, color='green', label='10-day ROC')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.7)
plt.axhline(y=5, color='orange', linestyle=':', linewidth=1, alpha=0.5, label='Overbought (+5%)')
plt.axhline(y=-5, color='orange', linestyle=':', linewidth=1, alpha=0.5, label='Oversold (-5%)')
plt.fill_between(maybank_roc_10.index, 0, maybank_roc_10.values, 
                 where=(maybank_roc_10 >= 0), alpha=0.3, color='green', label='Positive momentum')
plt.fill_between(maybank_roc_10.index, 0, maybank_roc_10.values, 
                 where=(maybank_roc_10 < 0), alpha=0.3, color='red', label='Negative momentum')
plt.title('10-Day Rate of Change (ROC)', fontsize=12, fontweight='bold')
plt.ylabel('ROC (%)')
plt.legend(loc='best', fontsize=8)
plt.grid(True, alpha=0.3)

# Subplot 3: Multiple ROC periods comparison
plt.subplot(3, 1, 3)
plt.plot(maybank_roc_10.index, maybank_roc_10.values, linewidth=1.5, label='10-day ROC', alpha=0.8)
plt.plot(maybank_roc_20.index, maybank_roc_20.values, linewidth=1.5, label='20-day ROC', alpha=0.8)
plt.plot(maybank_roc_30.index, maybank_roc_30.values, linewidth=1.5, label='30-day ROC', alpha=0.8)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.7)
plt.title('ROC Comparison - Different Periods', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('ROC (%)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observations:")
print("• Shorter periods (10-day) = More sensitive, more volatility")
print("• Longer periods (30-day) = Smoother, less noise")
print("• Zero line = Neutral momentum (price same as N days ago)")
print("• Above zero = Upward momentum")
print("• Below zero = Downward momentum")

### ROC vs Daily Returns

How is ROC different from daily returns?

- **Daily Returns**: Change from yesterday to today (1-day lookback)
- **ROC**: Change from N days ago to today (customizable lookback)

ROC is essentially a **multi-period return**.

In [ ]:
# Compare daily returns vs 10-day ROC
maybank_daily_returns = maybank_close.pct_change() * 100

print("Comparison: Daily Returns vs 10-Day ROC\n")
print("=" * 70)

print(f"\nDaily Returns:")
print(f"  Mean:   {maybank_daily_returns.mean():.4f}%")
print(f"  Std:    {maybank_daily_returns.std():.4f}%")
print(f"  Range:  {maybank_daily_returns.min():.2f}% to {maybank_daily_returns.max():.2f}%")

print(f"\n10-Day ROC:")
print(f"  Mean:   {maybank_roc_10.mean():.4f}%")
print(f"  Std:    {maybank_roc_10.std():.4f}%")
print(f"  Range:  {maybank_roc_10.min():.2f}% to {maybank_roc_10.max():.2f}%")

print(f"\n" + "=" * 70)
print("Notice:")
print("• ROC has larger std deviation (more volatile)")
print("• ROC has wider range (captures bigger moves)")
print("• Both measure momentum, but at different time scales")

---

## Part 3: Ratios - Comparing Two Values

### What is a Ratio?

A **ratio** compares two quantities by division.

### Formula

$$
\text{Ratio} = \frac{\text{Value A}}{\text{Value B}}
$$

### Common Ratios in Trading

1. **Gain/Loss Ratio** - Foundation for RSI
2. **High/Low Ratio** - Volatility measure
3. **Volume Ratio** - Comparing current vs average volume
4. **Price/MA Ratio** - How far price is from moving average

In [ ]:
# Example: Calculate gain/loss ratio (foundation for RSI)
maybank_returns = maybank_close.pct_change()

# Separate gains and losses
gains = maybank_returns.where(maybank_returns > 0, 0)
losses = -maybank_returns.where(maybank_returns < 0, 0)  # Make positive

# Calculate 14-period average gains and losses
period = 14
avg_gain = gains.rolling(window=period).mean()
avg_loss = losses.rolling(window=period).mean()

# Calculate gain/loss ratio
gain_loss_ratio = avg_gain / avg_loss

print("Gain/Loss Ratio Calculation (Foundation for RSI)\n")
print("=" * 70)

# Show example calculation for a specific date
example_idx = -1
example_date = maybank_close.index[example_idx]

print(f"\nDate: {example_date.strftime('%Y-%m-%d')}")
print(f"\nAverage gain (14-day):  {avg_gain.iloc[example_idx]:.6f}")
print(f"Average loss (14-day):  {avg_loss.iloc[example_idx]:.6f}")
print(f"\nGain/Loss Ratio = {avg_gain.iloc[example_idx]:.6f} / {avg_loss.iloc[example_idx]:.6f}")
print(f"                = {gain_loss_ratio.iloc[example_idx]:.4f}")

print(f"\nInterpretation:")
if gain_loss_ratio.iloc[example_idx] > 1:
    print(f"  Ratio > 1 → Average gains exceed average losses (bullish)")
elif gain_loss_ratio.iloc[example_idx] < 1:
    print(f"  Ratio < 1 → Average losses exceed average gains (bearish)")
else:
    print(f"  Ratio = 1 → Gains and losses are balanced (neutral)")

In [ ]:
# Visualize gain/loss ratio over time
plt.figure(figsize=(14, 8))

# Subplot 1: Price
plt.subplot(2, 1, 1)
plt.plot(maybank_close.index, maybank_close.values, linewidth=1.5, color='blue')
plt.title('Maybank Price (2023)', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

# Subplot 2: Gain/Loss Ratio
plt.subplot(2, 1, 2)
plt.plot(gain_loss_ratio.index, gain_loss_ratio.values, linewidth=1.5, color='purple')
plt.axhline(y=1, color='red', linestyle='--', linewidth=2, label='Neutral (Ratio = 1)')
plt.fill_between(gain_loss_ratio.index, 1, gain_loss_ratio.values,
                 where=(gain_loss_ratio >= 1), alpha=0.3, color='green', label='Gains > Losses')
plt.fill_between(gain_loss_ratio.index, 1, gain_loss_ratio.values,
                 where=(gain_loss_ratio < 1), alpha=0.3, color='red', label='Losses > Gains')
plt.title('14-Period Gain/Loss Ratio', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Ratio')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("This gain/loss ratio is the core component of RSI!")
print("We'll transform it into 0-100 scale next.")

---

## Part 4: Normalization - Scaling to 0-100

### What is Normalization?

**Normalization** transforms values to a standard scale (typically 0-100).

### Why Normalize?

1. **Easy interpretation** - Everyone understands 0-100 scale
2. **Comparison** - Different indicators on same scale
3. **Clear boundaries** - Overbought/oversold levels (70/30, 80/20)
4. **Visual clarity** - Charts are easier to read

### Min-Max Normalization Formula

$$
\text{Normalized Value} = \frac{\text{Value} - \text{Min}}{\text{Max} - \text{Min}} \times 100
$$

This scales any range to 0-100.

In [ ]:
# Example: Normalize stock prices to 0-100
maybank_close = maybank['Close']

# Calculate over a rolling 20-day window
period = 20

# Get rolling min and max
rolling_min = maybank_close.rolling(window=period).min()
rolling_max = maybank_close.rolling(window=period).max()

# Normalize using min-max formula
normalized_price = ((maybank_close - rolling_min) / (rolling_max - rolling_min)) * 100

print("Min-Max Normalization Example\n")
print("=" * 70)

# Show calculation for a specific day
example_idx = -1
example_date = maybank_close.index[example_idx]

print(f"\nDate: {example_date.strftime('%Y-%m-%d')}")
print(f"\nCurrent price:     RM {maybank_close.iloc[example_idx]:.4f}")
print(f"20-day lowest:     RM {rolling_min.iloc[example_idx]:.4f}")
print(f"20-day highest:    RM {rolling_max.iloc[example_idx]:.4f}")
print(f"\nFormula: (Current - Min) / (Max - Min) × 100")
print(f"       = ({maybank_close.iloc[example_idx]:.4f} - {rolling_min.iloc[example_idx]:.4f}) / ({rolling_max.iloc[example_idx]:.4f} - {rolling_min.iloc[example_idx]:.4f}) × 100")
print(f"       = {normalized_price.iloc[example_idx]:.2f}")

print(f"\nInterpretation:")
print(f"  0 = Price at 20-day low")
print(f"  100 = Price at 20-day high")
print(f"  {normalized_price.iloc[example_idx]:.1f} = Price is at {normalized_price.iloc[example_idx]:.1f}% of the 20-day range")

In [ ]:
# Visualize normalized price
plt.figure(figsize=(14, 10))

# Subplot 1: Original price
plt.subplot(3, 1, 1)
plt.plot(maybank_close.index, maybank_close.values, linewidth=1.5, color='blue')
plt.title('Original Price', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

# Subplot 2: Normalized price (0-100)
plt.subplot(3, 1, 2)
plt.plot(normalized_price.index, normalized_price.values, linewidth=1.5, color='purple')
plt.axhline(y=80, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Overbought (80)')
plt.axhline(y=20, color='green', linestyle='--', linewidth=1, alpha=0.7, label='Oversold (20)')
plt.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.5, label='Midpoint (50)')
plt.fill_between(normalized_price.index, 80, 100, alpha=0.2, color='red')
plt.fill_between(normalized_price.index, 0, 20, alpha=0.2, color='green')
plt.title('Normalized Price (0-100 Scale)', fontsize=12, fontweight='bold')
plt.ylabel('Normalized Value')
plt.ylim(-5, 105)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# Subplot 3: Both overlaid (scaled)
plt.subplot(3, 1, 3)
# Normalize original price to 0-100 for entire period for comparison
full_norm = ((maybank_close - maybank_close.min()) / (maybank_close.max() - maybank_close.min())) * 100
plt.plot(full_norm.index, full_norm.values, linewidth=1.5, label='Full period normalization', alpha=0.7)
plt.plot(normalized_price.index, normalized_price.values, linewidth=1.5, label='Rolling 20-day normalization', alpha=0.7)
plt.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.5)
plt.title('Comparison: Rolling vs Full Period Normalization', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Normalized Value')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice:")
print("• Rolling normalization adapts to recent price action")
print("• This is exactly how Stochastic Oscillator works!")
print("• Above 80 = Potentially overbought")
print("• Below 20 = Potentially oversold")

### Stochastic Oscillator Formula

The **Stochastic Oscillator** is exactly this normalization:

$$
\%K = \frac{\text{Close} - \text{Low}_{14}}{\text{High}_{14} - \text{Low}_{14}} \times 100
$$

Where:
- $\text{Low}_{14}$ = Lowest price in last 14 periods
- $\text{High}_{14}$ = Highest price in last 14 periods

**We just calculated the Stochastic Oscillator!**

---

## Part 5: RSI - Combining Ratios and Normalization

### What is RSI?

**Relative Strength Index (RSI)** combines gain/loss ratios with normalization.

### RSI Formula (Complete)

$$
\begin{aligned}
RS &= \frac{\text{Average Gain}}{\text{Average Loss}} \\
RSI &= 100 - \frac{100}{1 + RS}
\end{aligned}
$$

### Breaking Down the Formula

1. **Step 1**: Calculate average gains and losses (typically 14 periods)
2. **Step 2**: Calculate RS (Relative Strength) = Avg Gain / Avg Loss
3. **Step 3**: Transform RS to 0-100 scale using the RSI formula

### Why This Formula?

The formula $100 - \frac{100}{1 + RS}$ has special properties:
- When RS = 0 (no gains) → RSI = 0
- When RS = 1 (equal gains/losses) → RSI = 50
- When RS → ∞ (all gains) → RSI → 100
- RSI is always between 0 and 100

In [ ]:
# Calculate RSI from scratch
def calculate_rsi(prices, period=14):
    """
    Calculate RSI indicator from scratch.
    
    Parameters:
    -----------
    prices : pd.Series
        Price data (typically closing prices)
    period : int
        Lookback period (default 14)
    
    Returns:
    --------
    pd.Series
        RSI values (0-100)
    """
    # Calculate price changes
    delta = prices.diff()
    
    # Separate gains and losses
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses
    avg_gains = gains.rolling(window=period).mean()
    avg_losses = losses.rolling(window=period).mean()
    
    # Calculate RS (Relative Strength)
    rs = avg_gains / avg_losses
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Calculate RSI for Maybank
maybank_rsi = calculate_rsi(maybank_close, period=14)

print("RSI Calculation for Maybank\n")
print("=" * 70)
print(f"\nCurrent RSI (14-period): {maybank_rsi.iloc[-1]:.2f}")
print(f"\nLast 10 RSI values:")
print(maybank_rsi.tail(10))

In [ ]:
# Manual calculation example for one specific date
example_idx = -1
example_date = maybank_close.index[example_idx]

# Get the data we need
period = 14
delta = maybank_close.diff()
gains = delta.where(delta > 0, 0)
losses = -delta.where(delta < 0, 0)
avg_gain = gains.rolling(window=period).mean().iloc[example_idx]
avg_loss = losses.rolling(window=period).mean().iloc[example_idx]

# Calculate RS
rs = avg_gain / avg_loss

# Calculate RSI
rsi = 100 - (100 / (1 + rs))

print(f"\nManual RSI Calculation for {example_date.strftime('%Y-%m-%d')}:")
print("=" * 70)
print(f"\nStep 1: Calculate average gain and loss (14 periods)")
print(f"  Average Gain: {avg_gain:.6f}")
print(f"  Average Loss: {avg_loss:.6f}")

print(f"\nStep 2: Calculate RS (Relative Strength)")
print(f"  RS = Average Gain / Average Loss")
print(f"  RS = {avg_gain:.6f} / {avg_loss:.6f}")
print(f"  RS = {rs:.4f}")

print(f"\nStep 3: Calculate RSI")
print(f"  RSI = 100 - (100 / (1 + RS))")
print(f"  RSI = 100 - (100 / (1 + {rs:.4f}))")
print(f"  RSI = 100 - (100 / {1 + rs:.4f})")
print(f"  RSI = 100 - {100 / (1 + rs):.4f}")
print(f"  RSI = {rsi:.2f}")

print(f"\nVerification with our function: {maybank_rsi.iloc[example_idx]:.2f}")
print(f"Difference: {abs(rsi - maybank_rsi.iloc[example_idx]):.6f}")

In [ ]:
# Visualize RSI
plt.figure(figsize=(14, 10))

# Subplot 1: Price
plt.subplot(3, 1, 1)
plt.plot(maybank_close.index, maybank_close.values, linewidth=1.5, color='blue')
plt.title('Maybank Price (2023)', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

# Subplot 2: RSI
plt.subplot(3, 1, 2)
plt.plot(maybank_rsi.index, maybank_rsi.values, linewidth=2, color='purple', label='RSI (14)')
plt.axhline(y=70, color='red', linestyle='--', linewidth=2, label='Overbought (70)')
plt.axhline(y=30, color='green', linestyle='--', linewidth=2, label='Oversold (30)')
plt.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.5, label='Neutral (50)')
plt.fill_between(maybank_rsi.index, 70, 100, alpha=0.2, color='red')
plt.fill_between(maybank_rsi.index, 0, 30, alpha=0.2, color='green')
plt.title('RSI - Relative Strength Index (14-period)', fontsize=12, fontweight='bold')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# Subplot 3: RSI with signals
plt.subplot(3, 1, 3)
plt.plot(maybank_rsi.index, maybank_rsi.values, linewidth=2, color='purple')
plt.axhline(y=70, color='red', linestyle='--', linewidth=2, alpha=0.5)
plt.axhline(y=30, color='green', linestyle='--', linewidth=2, alpha=0.5)

# Highlight overbought and oversold conditions
overbought = maybank_rsi > 70
oversold = maybank_rsi < 30

plt.scatter(maybank_rsi[overbought].index, maybank_rsi[overbought].values, 
           color='red', s=30, alpha=0.6, label=f'Overbought ({overbought.sum()} days)')
plt.scatter(maybank_rsi[oversold].index, maybank_rsi[oversold].values, 
           color='green', s=30, alpha=0.6, label=f'Oversold ({oversold.sum()} days)')

plt.title('RSI with Overbought/Oversold Signals', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate statistics
print(f"\nRSI Statistics for Maybank (2023):")
print(f"  Mean RSI:        {maybank_rsi.mean():.2f}")
print(f"  Median RSI:      {maybank_rsi.median():.2f}")
print(f"  Max RSI:         {maybank_rsi.max():.2f}")
print(f"  Min RSI:         {maybank_rsi.min():.2f}")
print(f"\n  Days > 70 (overbought): {overbought.sum()} ({overbought.sum()/len(maybank_rsi)*100:.1f}%)")
print(f"  Days < 30 (oversold):   {oversold.sum()} ({oversold.sum()/len(maybank_rsi)*100:.1f}%)")
print(f"  Days 30-70 (neutral):   {((maybank_rsi >= 30) & (maybank_rsi <= 70)).sum()} ({((maybank_rsi >= 30) & (maybank_rsi <= 70)).sum()/len(maybank_rsi)*100:.1f}%)")

### Understanding the RSI Formula

Let's verify why the RSI formula works mathematically:

In [ ]:
# Test RSI formula with different RS values
print("Understanding RSI Formula: RSI = 100 - (100 / (1 + RS))\n")
print("=" * 70)

rs_values = [0, 0.5, 1, 2, 3, 5, 10, 50, 100]

print(f"\n{'RS':>8} | {'Interpretation':^30} | {'RSI':>6}")
print("-" * 70)

for rs in rs_values:
    if rs == 0:
        rsi = 0  # Special case: no gains
        interpretation = "No gains (all losses)"
    else:
        rsi = 100 - (100 / (1 + rs))
        if rs < 1:
            interpretation = "Losses > Gains"
        elif rs == 1:
            interpretation = "Gains = Losses"
        else:
            interpretation = "Gains > Losses"
    
    print(f"{rs:8.1f} | {interpretation:^30} | {rsi:6.2f}")

print("\n" + "=" * 70)
print("Key Insights:")
print("• RS = 0 (no gains) → RSI = 0")
print("• RS = 1 (equal gains/losses) → RSI = 50")
print("• RS > 1 (more gains) → RSI > 50")
print("• RS → ∞ (all gains) → RSI → 100")
print("• The formula automatically scales RS to 0-100 range!")

In [ ]:
# Visualize the RSI transformation function
rs_range = np.linspace(0, 10, 1000)
rsi_range = 100 - (100 / (1 + rs_range))

plt.figure(figsize=(12, 6))
plt.plot(rs_range, rsi_range, linewidth=3, color='purple')
plt.axhline(y=50, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='RSI = 50 (RS = 1)')
plt.axvline(x=1, color='gray', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=70, color='red', linestyle=':', linewidth=1, alpha=0.5, label='Overbought (70)')
plt.axhline(y=30, color='green', linestyle=':', linewidth=1, alpha=0.5, label='Oversold (30)')

# Mark special points
special_points = [(0, 0), (1, 50), (2, 100 - 100/3), (3, 75), (5, 100 - 100/6)]
for rs, rsi in special_points:
    plt.scatter([rs], [rsi], s=100, color='red', zorder=5, edgecolors='black', linewidths=2)
    plt.annotate(f'RS={rs}\nRSI={rsi:.1f}', xy=(rs, rsi), xytext=(rs+0.5, rsi+5),
                fontsize=9, ha='left')

plt.title('RSI Transformation Function: How RS Maps to RSI', fontsize=14, fontweight='bold')
plt.xlabel('RS (Relative Strength = Avg Gain / Avg Loss)', fontsize=12)
plt.ylabel('RSI (Relative Strength Index)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='lower right')
plt.xlim(0, 10)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()

print("Notice:")
print("• The curve is steeper near RS = 1 (around RSI = 50)")
print("• RSI becomes less sensitive as RS gets very large")
print("• This is why RSI rarely reaches extreme values (0 or 100)")

---

## Part 6: Exercises

Time to practice! Complete these exercises to master the concepts.

### Exercise 1: Calculate ROC for Multiple Periods

Calculate and compare ROC for Top Glove using 5, 10, and 20-day periods.

**Tasks**:
1. Calculate 5-day, 10-day, and 20-day ROC
2. Create a visualization comparing all three
3. Calculate which period had the highest volatility (std deviation)
4. Explain which period is best for short-term vs long-term trading

In [ ]:
# Your code here



<details>
<summary><b>Click here for solution</b></summary>

```python
# Calculate ROC for different periods
topglove_close = topglove['Close']

roc_5 = ((topglove_close - topglove_close.shift(5)) / topglove_close.shift(5)) * 100
roc_10 = ((topglove_close - topglove_close.shift(10)) / topglove_close.shift(10)) * 100
roc_20 = ((topglove_close - topglove_close.shift(20)) / topglove_close.shift(20)) * 100

# Visualize
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(topglove_close.index, topglove_close.values, linewidth=1.5, color='blue')
plt.title('Top Glove Price', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(roc_5.index, roc_5.values, linewidth=1.5, label='5-day ROC', alpha=0.8)
plt.plot(roc_10.index, roc_10.values, linewidth=1.5, label='10-day ROC', alpha=0.8)
plt.plot(roc_20.index, roc_20.values, linewidth=1.5, label='20-day ROC', alpha=0.8)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.5)
plt.title('Rate of Change Comparison', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('ROC (%)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate volatility
print("ROC Volatility (Standard Deviation):")
print(f"5-day ROC:  {roc_5.std():.4f}%")
print(f"10-day ROC: {roc_10.std():.4f}%")
print(f"20-day ROC: {roc_20.std():.4f}%")

print("\nInterpretation:")
print("• Shorter periods = More sensitive, higher volatility")
print("• Best for short-term: 5-day ROC (catches quick moves)")
print("• Best for long-term: 20-day ROC (filters out noise)")
```
</details>

### Exercise 2: Build a Stochastic Oscillator

Create a complete Stochastic Oscillator (%K and %D) from scratch.

**Tasks**:
1. Calculate %K using 14-period min-max normalization
2. Calculate %D as 3-period SMA of %K
3. Visualize both lines with overbought/oversold levels
4. Identify crossover signals (when %K crosses %D)

In [ ]:
# Your code here



<details>
<summary><b>Click here for solution</b></summary>

```python
# Calculate Stochastic Oscillator
cimb_close = cimb['Close']
period = 14

# %K: Normalize price position
low_14 = cimb_close.rolling(window=period).min()
high_14 = cimb_close.rolling(window=period).max()
stoch_k = ((cimb_close - low_14) / (high_14 - low_14)) * 100

# %D: 3-period SMA of %K
stoch_d = stoch_k.rolling(window=3).mean()

# Visualize
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(cimb_close.index, cimb_close.values, linewidth=1.5, color='blue')
plt.title('CIMB Price', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(stoch_k.index, stoch_k.values, linewidth=1.5, label='%K (14)', color='blue')
plt.plot(stoch_d.index, stoch_d.values, linewidth=2, label='%D (3-period SMA of %K)', color='red')
plt.axhline(y=80, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Overbought (80)')
plt.axhline(y=20, color='green', linestyle='--', linewidth=1, alpha=0.7, label='Oversold (20)')
plt.fill_between(stoch_k.index, 80, 100, alpha=0.2, color='red')
plt.fill_between(stoch_k.index, 0, 20, alpha=0.2, color='green')

# Identify crossovers
bullish_cross = (stoch_k > stoch_d) & (stoch_k.shift(1) <= stoch_d.shift(1))
bearish_cross = (stoch_k < stoch_d) & (stoch_k.shift(1) >= stoch_d.shift(1))

plt.scatter(stoch_k[bullish_cross].index, stoch_k[bullish_cross].values,
           marker='^', s=100, color='green', label='Bullish crossover', zorder=5)
plt.scatter(stoch_k[bearish_cross].index, stoch_k[bearish_cross].values,
           marker='v', s=100, color='red', label='Bearish crossover', zorder=5)

plt.title('Stochastic Oscillator (%K and %D)', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Stochastic Value')
plt.ylim(0, 100)
plt.legend(loc='best', fontsize=9)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Bullish crossovers: {bullish_cross.sum()}")
print(f"Bearish crossovers: {bearish_cross.sum()}")
```
</details>

### Exercise 3: Compare RSI Across Stocks

Calculate and compare RSI for all three stocks (Maybank, Top Glove, CIMB).

**Tasks**:
1. Calculate 14-period RSI for all three stocks
2. Create a comparison chart showing all three RSIs
3. Calculate statistics (mean, time in overbought/oversold)
4. Determine which stock is most overbought/oversold currently

In [ ]:
# Your code here



<details>
<summary><b>Click here for solution</b></summary>

```python
# Calculate RSI for all three stocks
maybank_rsi = calculate_rsi(maybank['Close'], 14)
topglove_rsi = calculate_rsi(topglove['Close'], 14)
cimb_rsi = calculate_rsi(cimb['Close'], 14)

# Visualize comparison
plt.figure(figsize=(14, 6))

plt.plot(maybank_rsi.index, maybank_rsi.values, linewidth=2, label='Maybank', alpha=0.8)
plt.plot(topglove_rsi.index, topglove_rsi.values, linewidth=2, label='Top Glove', alpha=0.8)
plt.plot(cimb_rsi.index, cimb_rsi.values, linewidth=2, label='CIMB', alpha=0.8)

plt.axhline(y=70, color='red', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=30, color='green', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.3)
plt.fill_between(maybank_rsi.index, 70, 100, alpha=0.1, color='red')
plt.fill_between(maybank_rsi.index, 0, 30, alpha=0.1, color='green')

plt.title('RSI Comparison - Malaysian Stocks (2023)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Statistics
stocks_data = {
    'Maybank': maybank_rsi,
    'Top Glove': topglove_rsi,
    'CIMB': cimb_rsi
}

print("\nRSI Statistics Comparison:\n")
print(f"{'Stock':<12} | {'Current':>8} | {'Mean':>8} | {'Overbought':>12} | {'Oversold':>10}")
print("-" * 70)

for stock_name, rsi in stocks_data.items():
    current = rsi.iloc[-1]
    mean = rsi.mean()
    overbought_pct = (rsi > 70).sum() / len(rsi) * 100
    oversold_pct = (rsi < 30).sum() / len(rsi) * 100
    
    print(f"{stock_name:<12} | {current:8.2f} | {mean:8.2f} | {overbought_pct:11.1f}% | {oversold_pct:9.1f}%")

# Current status
print("\nCurrent Status:")
for stock_name, rsi in stocks_data.items():
    current = rsi.iloc[-1]
    if current > 70:
        status = "OVERBOUGHT"
    elif current < 30:
        status = "OVERSOLD"
    else:
        status = "Neutral"
    print(f"{stock_name}: {current:.2f} ({status})")
```
</details>

### Exercise 4: Custom Normalization

Create a custom indicator that normalizes daily volume to 0-100 scale.

**Tasks**:
1. Calculate 20-day rolling min and max volume for Maybank
2. Normalize current volume to 0-100 scale
3. Plot volume and normalized volume
4. Identify days with "extreme" volume (> 80 or < 20)

In [ ]:
# Your code here



<details>
<summary><b>Click here for solution</b></summary>

```python
# Get volume data
maybank_volume = maybank['Volume']

# Calculate rolling min/max
period = 20
volume_min = maybank_volume.rolling(window=period).min()
volume_max = maybank_volume.rolling(window=period).max()

# Normalize volume
normalized_volume = ((maybank_volume - volume_min) / (volume_max - volume_min)) * 100

# Visualize
plt.figure(figsize=(14, 10))

# Original volume
plt.subplot(3, 1, 1)
plt.plot(maybank_close.index, maybank_close.values, linewidth=1.5, color='blue')
plt.title('Maybank Price', fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.bar(maybank_volume.index, maybank_volume.values, width=1, color='gray', alpha=0.7)
plt.title('Trading Volume (Original)', fontsize=12, fontweight='bold')
plt.ylabel('Volume')
plt.grid(True, alpha=0.3, axis='y')

# Normalized volume
plt.subplot(3, 1, 3)
colors = ['red' if v > 80 else 'green' if v < 20 else 'gray' for v in normalized_volume]
plt.bar(normalized_volume.index, normalized_volume.values, width=1, color=colors, alpha=0.7)
plt.axhline(y=80, color='red', linestyle='--', linewidth=2, alpha=0.7, label='High volume threshold (80)')
plt.axhline(y=20, color='green', linestyle='--', linewidth=2, alpha=0.7, label='Low volume threshold (20)')
plt.title('Normalized Volume (0-100 Scale, 20-day)', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Normalized Volume')
plt.ylim(0, 105)
plt.legend(loc='best')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Identify extreme volume days
high_volume = normalized_volume > 80
low_volume = normalized_volume < 20

print(f"\nExtreme Volume Days:")
print(f"High volume days (>80): {high_volume.sum()} ({high_volume.sum()/len(normalized_volume)*100:.1f}%)")
print(f"Low volume days (<20): {low_volume.sum()} ({low_volume.sum()/len(normalized_volume)*100:.1f}%)")
print(f"\nMost recent high volume days:")
print(normalized_volume[high_volume].tail(5))
```
</details>

---

## Summary

Congratulations! You've completed Module 03. Let's review what you mastered:

### Key Concepts Mastered

1. **Percentage Changes**
   - Formula: (New - Old) / Old × 100%
   - Not symmetric: +10% then -10% ≠ break even
   - Larger gains needed to recover from losses

2. **Rate of Change (ROC)**
   - Momentum indicator measuring percentage change over N periods
   - Formula: (Price - Price_N_days_ago) / Price_N_days_ago × 100%
   - Shorter periods = more sensitive
   - Longer periods = smoother

3. **Ratios**
   - Comparing two values by division
   - Gain/Loss ratio is foundation for RSI
   - Ratio > 1 = numerator larger
   - Ratio < 1 = denominator larger

4. **Normalization (Min-Max)**
   - Formula: (Value - Min) / (Max - Min) × 100
   - Scales any range to 0-100
   - Foundation for Stochastic Oscillator
   - Used for overbought/oversold indicators

5. **RSI (Relative Strength Index)**
   - Step 1: Calculate avg gains and losses
   - Step 2: Calculate RS = Avg Gain / Avg Loss
   - Step 3: RSI = 100 - (100 / (1 + RS))
   - RSI > 70 = Overbought
   - RSI < 30 = Oversold

### How This Connects to Technical Indicators

You now understand the math behind:
- **RSI**: Gain/loss ratio normalized to 0-100
- **Stochastic**: Price position normalized to 0-100
- **ROC**: Direct percentage change momentum
- **Williams %R**: Similar to Stochastic but scaled -100 to 0
- **MFI (Money Flow Index)**: Volume-weighted RSI

### What's Next?

In **Module 04: Moving Averages - The Complete Math**, you'll learn:
- Simple Moving Average (SMA) - detailed calculation
- Exponential Moving Average (EMA) - the weighting formula
- Why EMA reacts faster than SMA (mathematically)
- Smoothing constants and their effects
- How moving averages filter noise

### Additional Practice

Before Module 04, try:
1. Calculate RSI for different periods (7, 14, 21) and compare
2. Create custom oscillators using different normalization methods
3. Combine RSI with ROC to create a multi-indicator signal
4. Test RSI overbought/oversold signals on historical data

---

## Additional Resources

### Further Reading
- [Investopedia: RSI](https://www.investopedia.com/terms/r/rsi.asp)
- [Investopedia: Rate of Change (ROC)](https://www.investopedia.com/terms/r/rateofchange.asp)
- [Investopedia: Stochastic Oscillator](https://www.investopedia.com/terms/s/stochasticoscillator.asp)
- [StockCharts: RSI](https://school.stockcharts.com/doku.php?id=technical_indicators:relative_strength_index_rsi)

### Technical Papers
- Wilder, J. Wells (1978). "New Concepts in Technical Trading Systems" - Original RSI paper

### Python Documentation
- [Pandas rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)
- [NumPy where](https://numpy.org/doc/stable/reference/generated/numpy.where.html)

---

**Excellent work!** You've now mastered the mathematical foundations of the most popular oscillator indicators. Ready for Module 04?